In [46]:
### Installing packages
# pip install selenium
# pip install webdrive_manager
# pip install yfinance

# Import packages
import yfinance as yf
import requests

from selenium import webdriver 
from selenium.webdriver.chrome.service import Service as ChromeService 
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager

### Strategy

The stock screening strategy below is designed to complement the VWAP and ABCD pattern day trading strategies outlined in the book "Day Trading for a Living" by

#### Pre-Market Gappers
Criteria for the Gappers Scanner includes:
- Stocks that in the pre-market gapped up or down at least 2%
- Stocks that have traded at least 100,000 shares by 9am in pre-market
- Stocks that have an average daily volume of over 500,000 shares
- Stocks that have Average True Range (ATR) of at least 50 cents
- There is a fundamental catalyst for the stock

#### Additional Filter
We will filter for only Medium Float stocks (20 to 500 million shares) with a price range between $10-$100 and Large float stocks (500+ million shares) with a price range > $20. These stock work best with the VWAP and ABCD pattern trading strategies.

#### Human Screen
I will review the output manually and do my own research into whether there is a fundamental catalyst for the stock


In [47]:
# Initialising Selenium and setting options
options = webdriver.ChromeOptions() 
options.headless = True 
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options) 

In [48]:
# Lets begin by webscraping a list of the top 100 losers and top 100 gainers for the day

# Gainers
t100_gainers_url = "https://finance.yahoo.com/screener/predefined/day_gainers?count=100&offset=0"

# Losers
t100_losers_url = "https://finance.yahoo.com/screener/predefined/day_losers?count=100&offset=0"

In [49]:
# Function to scrape list stock symbols from Yahoo Finance's screener list
def scrape_gappers(url, driver):
    # Load page
	driver.get(url)
    
	# Find row element
	row_eles = driver.find_elements(By.CLASS_NAME, 'simpTblRow')

	# In each row we are interested in the Ticker Symbol and Price. 
	# Although we will also be filtering for Volume and Float (Market Cap/Price), we will do this at a later stage once we search for the ticker symbols using the yfinance library. This will save us the trouble of having to parse text with B and M representing billions and millions.
	gappers_list = list(map(row_to_tuple, row_eles))
	return gappers_list

# Function to convert Yahoo Finance row element to a tuple of (Ticker Symbol, Price)
def row_to_tuple(row_ele):
	# Getting the Ticker Symbol
	symbol_ele = row_ele.find_element(By.XPATH,'td[@aria-label="Symbol"]')
	symbol_str = symbol_ele.text

	# Getting the Price
	price_ele = row_ele.find_element(By.XPATH,'td[@aria-label="Price (Intraday)"]')
	price_num = float(price_ele.text)

	# Returning data as a tuple - (Ticker Symbol: String, Price: Float)
	return (symbol_str, price_num)

In [50]:
# Stock ticker symbols
t100_gainers_all = scrape_gappers(t100_gainers_url, driver)
t100_losers_all = scrape_gappers(t100_losers_url, driver)

### One criteria of the screener is that the price much be above $10. Lets filter these down based on this.

# Function to filter Stocks with prices > $10 and return only the Ticker Symbol
def stocks_above_10(sym_pri_tuple_lst):
	SYMBOL_TUPLE_INDEX = 0
	PRICE_TUPLE_INDEX = 1

	stocks_above_10 = []

	for tuple in sym_pri_tuple_lst:
		price = tuple[PRICE_TUPLE_INDEX]

		if (price > 10):
			symbol = tuple[SYMBOL_TUPLE_INDEX]
			stocks_above_10.append(symbol)
	
	return stocks_above_10

# Filtered Lists
t100_gainers = stocks_above_10(t100_gainers_all)
t100_losers = stocks_above_10(t100_losers_all)

print(t100_gainers)
print(t100_losers)

['HCM', 'BGNE', 'NAAS', 'ATAT', 'AI', 'DLO', 'KRYS', 'LEGN', 'COLB', 'DEA', 'CTLT', 'LKNCY', 'CFLT', 'FLNC', 'LESL', 'ALNY', 'ITCI', 'SYM', 'WAL', 'BUR', 'ROKU', 'MDGL', 'SYIEY', 'FRC', 'FOXF', 'RLAY', 'BABA', 'IMCR', 'TECK', 'ENV', 'RXDX', 'RYAAY', 'EWBC', 'EDR', 'ARGX', 'GOOGL', 'TTD', 'GOOG', 'PARAA', 'MRTX', 'DOCS', 'SRPT', 'ARWR', 'HKXCY', 'MNDY', 'GDS', 'CMA', 'SUPN', 'BPMC', 'PINS', 'DRS', 'SHTDY', 'HTHT', 'IAS', 'HSHCY', 'APPN', 'NEWR', 'TAK', 'FIS', 'PARA', 'BMRN', 'VIPS', 'BDRBF', 'SRAD', 'MPNGY', 'VIR']
['LEVI', 'LITE', 'PBI-PB', 'FN', 'SMCI', 'ZGN', 'MOS', 'CVCO', 'TTNDF', 'ABNB', 'NTR', 'GTLS', 'WIRE', 'VRT', 'ATKR', 'FMC', 'PBF', 'CPRI', 'UNVGY', 'SIG', 'RPM', 'KBH', 'SPWR', 'ARCH', 'KMTUY', 'BDC', 'SWAV', 'CRK', 'FRO', 'DKILY', 'WCC', 'ARW', 'MDC', 'FJTSY', 'CVE', 'CEQP', 'OLK', 'PCVX', 'DNUT', 'MPC', 'WIX', 'LSRCY', 'AIT', 'PEN', 'LECO', 'SANM', 'NEOG', 'PVH', 'LRLCY', 'BE', 'LGIH', 'TOELY', 'YPF', 'KTB', 'CIEN', 'MTSI', 'PKX']


In [74]:
# Great! Now that we have our list of gapping stocks above $10, lets get more detailed information about these with the yfinance package.
# We want information in the following data structure (Ticker Symbol: String, Price: float, Change: float, AverageVolume: int, Float: float, AverageTrueRange: float)

# Function to convert ticker symbol to (Ticker Symbol: String, Name: String, Price: float, Change: float, AverageVolume: int, Float: float, AverageTrueRange: float) data format
def tickers_to_data(tickers, data):
	result = []
        
	for ticker in tickers:
		ticker_data = data.tickers[ticker]

		### Getting data
		info = ticker_data.info
		
		name = info['longName']
		price = info['postMarketPrice']
		change = info['regularMarketChangePercent']
		average_vol = info['averageDailyVolume10Day']
		# Float Calculations
		market_cap = info['marketCap']
		ticker_float = market_cap/price
		# Average True Range - Substitute for day range
		day_range = info['regularMarketDayRange']
		min_max = day_range.split(' - ')
		print(min_max)
		range = abs(float(min_max[0]) - float(min_max[1]))

		### Apply Filters
		# Needs to be at least medium float (> 20 million shares)
		float_satisfied = ticker_float > 20,000,000
		# Needs to have an average volume of > 500,000
		avg_vol_satisfied = average_vol > 500,000
		# Needs to have a range of > $0.50
		range_satisfied = range > 0.5

		if (float_satisfied and avg_vol_satisfied and range_satisfied):
			ticker_res = (ticker, name, price, change, average_vol, ticker_float, range)
			result.append(ticker_res)
		
		# news = ticker_data.news

	return result
	


In [75]:
all_gappers = t100_gainers + t100_losers
data = yf.Tickers(all_gappers)

result = tickers_to_data(all_gappers, data)

print(result)

['13.9', '14.89']
['218.26', '242.38']
['10.95', '12.78']
['24.24', '26.2']
['20.4', '23.86']
['12.35', '14.09']
['77.865', '84.66']
['48.2', '51.31']
['20.92', '22.27']
['13.295', '14.42']
['60.9', '65.635']


KeyError: 'postMarketPrice'

In [91]:
hmc = yf.Ticker('AAPL')

# hmc.history().iloc[0,3]
hmc.history(interval='5m')

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Datetime,,,,,,,
2023-03-07 09:30:00-05:00,153.699997,153.830002,153.149994,153.429993,2751998,0.0,0.0
2023-03-07 09:35:00-05:00,153.421494,154.029907,153.384995,153.710007,1290787,0.0,0.0
2023-03-07 09:40:00-05:00,153.705002,153.850006,153.550003,153.600006,757485,0.0,0.0
2023-03-07 09:45:00-05:00,153.597504,153.910004,153.449997,153.634995,753449,0.0,0.0
2023-03-07 09:50:00-05:00,153.634995,153.919998,153.538696,153.869904,578911,0.0,0.0
...,...,...,...,...,...,...,...
2023-04-06 15:35:00-04:00,164.725006,164.759903,164.604996,164.619995,370977,0.0,0.0
2023-04-06 15:40:00-04:00,164.610001,164.610001,164.500000,164.554993,469346,0.0,0.0
2023-04-06 15:45:00-04:00,164.554993,164.559998,164.380005,164.475006,533176,0.0,0.0
